# Recommendations Systems
## Homework 3: Neural Collaborative Filtering

Submit your solution in the form of an Jupyter notebook file (with extension ipynb).   
Images of graphs or tables should be submitted as PNG or JPG files.   
The code used to answer the questions should be included, runnable and documented in the notebook.   
Python 3.6 should be used.

The goal of this homework is to let you understand the concept of  recommendations based on implicit data which is very common in real life, and learn how ‘Deep neural networks’ components can be used to implement a collaborative filtering and hybrid approach recommenders.  
Implementation example is presented in the <a href='https://colab.research.google.com/drive/1v72_zpCObTFMbNnQXUknoQVXR1vBRX6_?usp=sharing'>NeuralCollaborativeFiltering_Implicit</a> notebook in Moodle.

We will use a dataset based on the <a href='https://grouplens.org/datasets/movielens/1m/'>MovieLens 1M rating dataset</a> after some pre-processing to adapt it to an implicit feedback use case scenario.  
You can download the dataset used by <a href='https://github.com/hexiangnan/neural_collaborative_filtering'>this implementation</a> of the paper Neural Collaborative Filtering or from the NeuralCollaborativeFiltering_implicit notebook in Moodle.
<br>

## Imports:

In [43]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

from keras.layers import Embedding, Input, Dense, Reshape,  Flatten, Dropout
from keras.regularizers import l2
from keras import backend as K
from keras import initializers
from keras.initializers import RandomNormal
from keras.models import Sequential, Model, load_model, save_model
from keras.layers.core import Dense, Lambda, Activation
from keras.optimizers import Adagrad, Adam, SGD, RMSprop
from keras.layers import Multiply, Concatenate

# from time import time
# import multiprocessing as mp
# import sys
# import math
# import argparse

#### Constants:

#### Preprocessing:

In [23]:
# !git clone https://github.com/hexiangnan/neural_collaborative_filtering.git

In [40]:
column_names = ['user_id', 'item_id', 'rating', 'timestamp']

# Read the training file
training = pd.read_csv('./neural_collaborative_filtering/Data/ml-1m.train.rating', sep='\t', names=column_names)

# Read the test file
test_rating = pd.read_csv('./neural_collaborative_filtering/Data/ml-1m.test.rating', sep='\t', names=column_names)


negative_ids = ['(user_id, item_id)']

for i in range(1,100):
    negative_ids.append(f'id-{i}')

test_negative = pd.read_csv('./neural_collaborative_filtering/Data/ml-1m.test.negative', sep='\t', names=negative_ids)

In [41]:
training.loc[(training['user_id'] == 0) & (training['item_id'] == 8)]

,user_id,item_id,rating,timestamp
13,0,8,4,978302268


## Question 1: Dataset preparation

a. This implementation contains one file for training and two files for testing:
- ml-1m.train.rating
- ml-1m.test.rating
- ml-1m.test.negative

**Explain** the role and structure of each file and how it was created from the original MovieLens 1M rating dataset.

##### Answer:

ml-1m.train.rating:
- Training file
- Each line is a training instance: userID\t itemID\t rating\t timestamp (if exists)
- 1 million ratings, where each user has at least 20 ratings
- Similar to the training data from previous HWs

ml-1m.test.rating:
- Test file (positive instances)
- Each line is a testing instance: userID\t itemID\t rating\t timestamp (if exists)

ml-1m.test.negative:
- Test file (negative instances)
- Each line corresponds to the line of test.rating, containing 99 negative samples
- Each line is in the format: (userID,itemID)\t negativeItemID1\t negativeItemID2...

b. **Explain** how the training dataset is created.

##### Answer:

c. **Explain** how the test dataset is created.

##### Answer:

<br>

***
## Question 2: Neural Collaborative filtering

In [66]:
########## Maybe need to change like the "maybe" file
def get_train_instances(train, num_negatives):
    user_input, item_input, labels = [],[],[]
    num_users = train.shape[0]
    
    for (u, i) in train.keys():
        # positive instance
        user_input.append(u)
        item_input.append(i)
        labels.append(1)
        
        # negative instances
        for t in range(num_negatives):
            j = np.random.randint(num_items)
            
            while (u, j) in train:
                j = np.random.randint(num_items)
                
            user_input.append(u)
            item_input.append(j)
            labels.append(0)
    
    return user_input, item_input, labelsem_input, labels

a. Build the following four models using the neural collaborative filtering approach: 
- Matrix Factorization (MF)
- Multi layer perceptron (MLP)
- Generalized Matrix Factorization (GMF) 
- NeuroMatrixFactorization (NMF)

##### Answer:

In [ ]:
# Add Matrix Factorization (from previous HWs?)
# no need to use it in the assignment anyway. see 2b

In [61]:
def get_GMF_model(num_users, num_items, latent_dim, regs=[[0,0]], activation='sigmoid'):
    # Generalized Matrix Factorization
    
    # Input variables
    user_input = Input(shape=(1,), dtype='int32', name = 'user_input')
    item_input = Input(shape=(1,), dtype='int32', name = 'item_input')

    MF_Embedding_User = Embedding(input_dim = num_users, output_dim = latent_dim, name = 'user_embedding',
                                   embeddings_regularizer = l2(regs[0][0]), input_length=1,embeddings_initializer=RandomNormal(mean=0.0, stddev=0.01)) #init = init_normal,
    MF_Embedding_Item = Embedding(input_dim = num_items, output_dim = latent_dim, name = 'item_embedding',
                                   embeddings_regularizer = l2(regs[0][1]), input_length=1,embeddings_initializer=RandomNormal(mean=0.0, stddev=0.01))  #init = init_normal, 
    
    # Crucial to flatten an embedding vector!
    user_latent = Flatten()(MF_Embedding_User(user_input))
    item_latent = Flatten()(MF_Embedding_Item(item_input))
    
    # Element-wise product of user and item embeddings 
    predict_vector = Multiply()([user_latent, item_latent]) #merge([user_latent, item_latent], mode = 'mul')
    
    # Final prediction layer
    prediction = Dense(1, activation=activation, kernel_initializer='lecun_uniform', name = 'prediction')(predict_vector)
    model = Model(inputs=[user_input, item_input], outputs=prediction)
    
    return model


def get_MLP_model(num_users, num_items, latent_dim, regs=[[0,0],0,0], layers = [20,10], activation='sigmoid'):
    # Multi-Layer Perceptron
    
    assert len(layers) + 1 == len(regs), 'the number of regs is equal to number of layers + the embedding layer'
    num_layer = len(layers) #Number of layers in the MLP
    # Input variables
    user_input = Input(shape=(1,), dtype='int32', name = 'user_input')
    item_input = Input(shape=(1,), dtype='int32', name = 'item_input')

    MLP_Embedding_User = Embedding(input_dim = num_users, output_dim = latent_dim, name = 'user_embedding',
                                   embeddings_regularizer = l2(regs[0][0]), input_length=1,embeddings_initializer=RandomNormal(mean=0.0, stddev=0.01)) #init = init_normal,
    MLP_Embedding_Item = Embedding(input_dim = num_items, output_dim = latent_dim, name = 'item_embedding',
                                   embeddings_regularizer = l2(regs[0][1]), input_length=1,embeddings_initializer=RandomNormal(mean=0.0, stddev=0.01)) #init = init_normal,
    
    # Crucial to flatten an embedding vector!
    user_latent = Flatten()(MLP_Embedding_User(user_input))
    item_latent = Flatten()(MLP_Embedding_Item(item_input))
    
    # Concatenation of embedding layers
    vector = Concatenate(axis=-1)([user_latent, item_latent])#merge([user_latent, item_latent], mode = 'concat')
    
    # MLP layers
    for idx in range(num_layer):
        layer = Dense(layers[idx], kernel_regularizer = l2(regs[idx+1]), activation='relu', name = 'layer%d' %idx)
        vector = layer(vector)
        
    # Final prediction layer
    prediction = Dense(1, activation=activation, kernel_initializer='lecun_uniform', name = 'prediction')(vector)
    model = Model(inputs=[user_input, item_input], outputs=prediction)
    
    return model


def get_NMF_model(num_users, num_items, latent_dim_GMF, latent_dim_MLP, reg_GMF=[[0,0]], regs_MLP=[[0,0],0,0], layers=[20,10], activation='sigmoid'):
    # Neural matrix factorization
    assert len(layers) + 1 == len(regs_MLP), 'the number of regs is equal to number of layers + the embedding layer'
    num_layer = len(layers) #Number of layers in the MLP

    # Input variables
    user_input = Input(shape=(1,), dtype='int32', name = 'user_input')
    item_input = Input(shape=(1,), dtype='int32', name = 'item_input')
    
    # Embedding layer
    MF_Embedding_User = Embedding(input_dim = num_users, output_dim = latent_dim_GMF, name = 'MF_user_embedding',
                                   embeddings_regularizer = l2(reg_GMF[0][0]), input_length=1,embeddings_initializer=RandomNormal(mean=0.0, stddev=0.01)) #init = init_normal,
    MF_Embedding_Item = Embedding(input_dim = num_items, output_dim = latent_dim_GMF, name = 'MF_item_embedding',
                                   embeddings_regularizer = l2(reg_GMF[0][1]), input_length=1,embeddings_initializer=RandomNormal(mean=0.0, stddev=0.01))  #init = init_normal, 
    
    MLP_Embedding_User = Embedding(input_dim = num_users, output_dim = latent_dim_MLP, name = 'MLP_user_embedding',
                                   embeddings_regularizer = l2(regs_MLP[0][0]), input_length=1,embeddings_initializer=RandomNormal(mean=0.0, stddev=0.01)) #init = init_normal,
    MLP_Embedding_Item = Embedding(input_dim = num_items, output_dim = latent_dim_MLP, name = 'MLP_item_embedding',
                                   embeddings_regularizer = l2(regs_MLP[0][1]), input_length=1,embeddings_initializer=RandomNormal(mean=0.0, stddev=0.01)) #init = init_normal,
    
    # MF part
    mf_user_latent = Flatten()(MF_Embedding_User(user_input))
    mf_item_latent = Flatten()(MF_Embedding_Item(item_input))
    mf_vector = Multiply()([mf_user_latent, mf_item_latent]) #merge([mf_user_latent, mf_item_latent], mode = 'mul') # element-wise multiply

    # MLP part
    mlp_user_latent = Flatten()(MLP_Embedding_User(user_input))
    mlp_item_latent = Flatten()(MLP_Embedding_Item(item_input))
    mlp_vector = Concatenate(axis=-1)([mlp_user_latent, mlp_item_latent])#merge([mlp_user_latent, mlp_item_latent], mode = 'concat')
    
    for idx in range(num_layer):
        layer =  Dense(layers[idx], kernel_regularizer = l2(regs_MLP[idx+1]), activation='tanh', name = 'layer%d' %idx)
        mlp_vector = layer(mlp_vector)

    # Concatenate MF and MLP parts
    predict_vector = Concatenate(axis=-1)([mf_vector, mlp_vector])
    
    # Final prediction layer
    prediction = Dense(1, activation=activation, kernel_initializer='lecun_uniform', name = "prediction")(predict_vector)    
    model = Model(inputs=[user_input, item_input], outputs=prediction)
    
    return model

In [ ]:
num_factors = 8 # size of embedding size. Can be split to 4 different params potentially.
num_negatives = 4 # how many negative samples per positive sample?
learning_rate = 0.001
epochs = 10
batch_size = 256
verbose = 1
write_model=False
topK = 10 # used to evaluate the model. Top K recommendations are used.
evaluation_threads = 1 
model_out_file = 'Pretrain/%s_GMF_%d_%d.h5' %(dataset, num_factors, time())

In [ ]:
# Build models
mlp_model = get_MLP_model(num_users, num_items, num_factors, regs = [[0,0],0,0,0], layers = [32,16,8])
gmf_model = get_GMF_model(num_users, num_items, num_factors, regs = [[0,0]])
nmf_model = get_NMF_model(num_users, num_items, latent_dim_GMF=num_factors, latent_dim_MLP=num_factors, reg_GMF=[[0,0]], regs_MLP=[[0,0],0,0,0], layers=[32,16,8])

mlp_model.compile(optimizer=Adam(lr=learning_rate), loss='binary_crossentropy')
gmf_model.compile(optimizer=Adam(lr=learning_rate), loss='binary_crossentropy')
nmf_model.compile(optimizer=Adam(lr=learning_rate), loss='binary_crossentropy')

b. Train and evaluate the recommendations accuracy of three models: 
- MF or GMF
- MLP
- NMF

Compare the learning curve and recommendations accuracy using NDCG and MRR metrics with cutoff values of 5 and 10.   
Discuss the comparison.

##### Answer:

Model Evaluation

In [46]:
import math
import heapq # for retrieval topK
import multiprocessing
from time import time
#from numba import jit, autojit


def evaluate_model(model, testRatings, testNegatives, K, num_thread):
    """
    Evaluate the performance (MRR, NDCG) of top-K recommendation
    Return: score of each test rating.
    """
    mrrs = []
    ndcgs = []
    
    for idx in range(len(_testRatings)):
        (mrr, ndcg) = eval_one_rating(idx)
        mrrs.append(mrr)
        ndcgs.append(ndcg)
    
    return (mrrs, ndcgs)


########## Maybe need to change like the "maybe" file
def eval_one_rating(idx):
    rating = _testRatings[idx]
    items = _testNegatives[idx]
    u = rating[0]
    gtItem = rating[1]
    items.append(gtItem)
    # Get prediction scores
    map_item_score = {}
    users = np.full(len(items), u, dtype = 'int32')
    predictions = _model.predict([users, np.array(items)], 
                                 batch_size=100, verbose=0)
    
    for i in range(len(items)):
        item = items[i]
        map_item_score[item] = predictions[i]
    
    items.pop()
    
    # Evaluate top rank list
    ranklist = heapq.nlargest(_K, map_item_score, key=map_item_score.get)
    mrr = getMRR(ranklist, gtItem)
    ndcg = getNDCG(ranklist, gtItem)
    
    return (mrr, ndcg)


def getMRR(ranklist, gtItem):
    for i in range(len(ranklist)):
        item = ranklist[i]
        
        if item == gtItem:
            return 1/(i+1)
    return 0


def getNDCG(ranklist, gtItem):
    for i in range(len(ranklist)):
        item = ranklist[i]
        
        if item == gtItem:
            return math.log(2) / math.log(i+2)
    return 0

In [ ]:
models = [['MLP', mlp_model], ['GMF', gmf_model], ['NMF', nmf_model]]

for model in models:
    # Init performance    
t1 = time()    mrrsts, ndcgs) = evaluate_model(mod[1]el, testRatings, testNegatives, topK, evaluation_threads    mrr
hr, ndcg = np.array(hits).mean(), np.array(ndcgs).mea
    n    ()
prfi{model[0]} nt('InMRR: H{mrr}%.4f, NDC{ndcg}]Time = {(time()-t1)/60}')1))


Init: HR = 0.0892, NDCG = 0.0402	 [238.8 s]


c. How the values of MRR and NDCG are differ from the results you got in the previous exercises which implemented the explicit recommendation approach. 
What are the difference in preparing the dataset for evaluation.

##### Answer:

d. How will you measure item similarity using the NeuMF model?

##### Answer:

<br>

***
## Question 3: Loss function

a. One of the enhancements presented in the Neural Collaborative Filtering paper is the usage of probabilistic activation function (the sigmoid) and binary cross entropy loss function.   

Select one of the models you implemented in question 2 and change the loss function to a Mean Squared Error and the activation function of the last layer to RELU.   

Train the model and evaluate it in a similar way to what you did in question 2. 
Compare the results and discuss.

In [60]:
model_3a = get_MLP_model(num_users, num_items, num_factors, regs = [[0,0],0,0,0], layers = [32,16,8], activation='relu')
model_3a.compile(optimizer=Adam(lr=learning_rate), loss='mse')
print(model_3a.summary())

NameError: name 'num_users' is not defined